<img src="https://github.com/FIUBA-Posgrado-Inteligencia-Artificial/procesamiento_lenguaje_natural/raw/main/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Word2vect


In [44]:
import numpy as np
import pandas as pd

In [2]:
def cosine_similarity(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * (np.linalg.norm(b)))

### Datos

In [3]:
corpus = np.array(['que dia es hoy', 'martes el dia de hoy es martes', 'martes muchas gracias'])

Documento 1 --> que dia es hoy \
Documento 2 --> martes el dia de hoy es martes \
Documento 3 --> martes muchas gracias

### 1 - Obtener el vocabulario del corpus (los términos utilizados)
- Cada documento transformarlo en una lista de términos
- Armar un vector de términos no repetidos de todos los documentos

In [37]:
def vocabulary(corpus):    
    l = []
    for document in corpus:
        doc_split = document.split()
        l.append(doc_split)    
    flat_list = [item for sublist in l for item in sublist]
    vocabulary = list(set(flat_list))
    return vocabulary

In [38]:
vocabulary(corpus)

['dia', 'muchas', 'que', 'hoy', 'de', 'gracias', 'martes', 'es', 'el']

### 2- OneHot encoding
Data una lista de textos, devolver una matriz con la representación oneHotEncoding de estos

In [52]:
def my_inefficient_OHE(lista):
    voc = vocabulary(lista)
    len_voc = len(voc)
    len_list = len(lista)
    
    mat = np.zeros((len_list,len_voc))
    
    for i, row in enumerate(lista):
        sentence = row.split()
        for j, col in enumerate(voc):
            mat[i,j] = voc[j] in sentence
    
    return pd.DataFrame(mat.astype(int), columns = voc)

In [67]:
t1 = 'cheems hola cheems'
t2 = 'el perrito cheems tiene amsiedad'

In [68]:
lista = [t1, t2]

In [90]:
my_inefficient_OHE(corpus)

,dia,muchas,que,hoy,de,gracias,martes,es,el
0,1,0,1,1,0,0,0,1,0
1,1,0,0,1,1,0,1,1,1
2,0,1,0,0,0,1,1,0,0


### 3- Vectores de frecuencia
Data una lista de textos, devolver una matriz con la representación de frecuencia de estos

In [70]:
def my_inefficient_frec(lista):
    voc = vocabulary(lista)
    len_voc = len(voc)
    len_list = len(lista)
    
    mat = np.zeros((len_list,len_voc))
    
    for i, row in enumerate(lista):
        sentence = row.split()
        for j, col in enumerate(voc):
            mat[i,j] = sentence.count(voc[j])
    
    return pd.DataFrame(mat.astype(int), columns = voc)

In [91]:
my_inefficient_frec(corpus)

,dia,muchas,que,hoy,de,gracias,martes,es,el
0,1,0,1,1,0,0,0,1,0
1,1,0,0,1,1,0,2,1,1
2,0,1,0,0,0,1,1,0,0


### 4- TF-IDF
Data una lista de textos, devolver una matriz con la representacion TFIDF

In [97]:
def my_inefficient_TFIDF(lista):
    voc = vocabulary(lista)
    len_voc = len(voc)
    len_list = len(lista)
    
    mat = np.zeros((len_list,len_voc))
    
    for i, row in enumerate(lista):
        sentence = row.split()
        for j, col in enumerate(voc):
            mat[i,j] = voc[j] in sentence
            
    mat = np.log10(len_list/np.sum(mat, axis = 0)) * mat
    
    return pd.DataFrame(mat, columns = voc)

In [98]:
my_inefficient_TFIDF(corpus)

,dia,muchas,que,hoy,de,gracias,martes,es,el
0,0.176091,0.000000,0.477121,0.176091,0.000000,0.000000,0.000000,0.176091,0.000000
1,0.176091,0.000000,0.000000,0.176091,0.477121,0.000000,0.176091,0.176091,0.477121
2,0.000000,0.477121,0.000000,0.000000,0.000000,0.477121,0.176091,0.000000,0.000000


### 5 - Comparación de documentos
Realizar una funcion que reciba el corpus y el índice de un documento y devuelva los documentos ordenados por la similitud coseno

In [168]:
def sorted_cosine_similarity(corpus, idx):
    similarity_vec = []
    sentence = []
    mat = np.array(my_inefficient_TFIDF(corpus))
    a = mat[idx]
    sentence_picked = corpus[idx].copy()
    
    # Elimino la fila que elegí como índice.
    mat = np.delete(mat, (idx), axis = 0)
    corpus = np.delete(corpus, (idx), axis = 0)
    for row in range(len(mat)):    
        b = mat[row]
        simi = cosine_similarity(a, b)
        sent = corpus[row]
        similarity_vec.append(simi)
        sentence.append(sent)
    
    MAT = pd.DataFrame({'Document': sentence, 'Cosine_Similarity': similarity_vec})
    MAT = MAT.sort_values('Cosine_Similarity', ascending = False)
    
    # Agrego como primera columna al documento escogido.
    MAT = np.array(MAT)
    MAT = np.vstack(([sentence_picked, 1], MAT))
    MAT = pd.DataFrame(MAT, columns = ['Document', 'Cosine_Similarity'])
    return MAT

In [169]:
sorted_cosine_similarity(corpus, 0)

,Document,Cosine_Similarity
0,que dia es hoy,1
1,martes el dia de hoy es martes,0.215828
2,martes muchas gracias,0.0
